<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/Introduction_in_NLP/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80_%D0%9A%D0%BB%D1%8E%D0%BA%D0%B8%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание:
1. Объяснить значение трех UD-тегов синтаксических отношений на примере фрагмента размеченного корпуса.
2. Написать функцию разбиения сложносочиненного предложения из двух частей на простые.
3. Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.
4. Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.


## задание 1 Ud-теги

In [10]:
! pip install natasha

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=bfe76d1ca59675f64239d58bcedb533c3fa938b97ffcee69fdc14aa53e066951
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=83c7773c4dd8d16516e9119fcdfb7012b9691416f3ca905f9cba74bfc08e802e
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built docopt intervaltree


In [3]:
from natasha import Segmenter, NewsEmbedding, NewsSyntaxParser, Doc

segmenter = Segmenter()
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

# tokens must be defined for this code to run.
# Based on the output tree, the tokens could be:

text = "Первое издание «Москва-Петушки» быстро разошлось."

# tokens =
# tokens = [
#     "Создатели", "уже", "подписали", "соглашение", "с", "рядом",
#     "игровых", "компаний", ",", "включая", "EA", "и", "Hi-Rez", "."
# ]

doc = Doc(text)
doc.segment(segmenter)
doc.parse_syntax(syntax_parser)
doc.sents[0].syntax.print()

    ┌► Первое         amod
┌►┌─└─ издание        nsubj
│ │ ┌► «              punct
│ └►└─ Москва-Петушки appos
│ └──► »              punct
│   ┌► быстро         advmod
└─┌─└─ разошлось      
  └──► .              punct


## Задание 2: разбиваем сложносочинённое предложение на простые
Написать функцию разбиения сложносочиненного предложения из двух частей на простые.

В русском языке зачастую это сочетание сс - союза и punct пунктуации

In [4]:
!pip install -U spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import spacy
nlp = spacy.load("ru_core_news_sm")

def split_compound_sentence(sentence):

    doc = nlp(sentence)
    simple_sentences = []
    split_index = -1


    for i, token in enumerate(doc):
        if token.dep_ == "cc" or token.dep_ == 'mark':
            split_index = i
            break

    if split_index != -1:

        sent1_tokens = doc[:split_index - 1]
        sentence1 = "".join(token.text_with_ws for token in sent1_tokens).strip()

        sent2_tokens = doc[split_index + 1:]
        sentence2 = "".join(token.text_with_ws for token in sent2_tokens).strip()
        # починить заглавную букву
        sentence1 = sentence1[0].upper() + sentence1[1:]
        sentence2 = sentence2[0].upper() + sentence2[1:]


        if not sentence1.endswith(('.', '!', '?')):
             sentence1 += '.'
        if not sentence2.endswith(('.', '!', '?')):
             sentence2 += '.'


        simple_sentences.append(sentence1)
        simple_sentences.append(sentence2)
    else:
        # Если союз не найден, возвращаем исходное предложение как одно целое
        simple_sentences.append(sentence)

    return simple_sentences

In [8]:
sentence = "Было тихо, и над деревней стоял теплый туман."


print(f"\nИсходное предложение: '{sentence}'")
split_sentences = split_compound_sentence(sentence)
print(f"Разбитые предложения: {split_sentences}")


Исходное предложение: 'Было тихо, и над деревней стоял теплый туман.'
Разбитые предложения: ['Было тихо.', 'Над деревней стоял теплый туман.']


## Задание 3: функция нахождения наименьшего предка
Написать функцию нахождения наименьшего общего предка двух токенов в дереве зависимостей.



In [11]:
text = "Прислонясь к дверному косяку я ловлю в далёком отголоске, что случиться на моём веку"
doc = nlp(text)
# print(token1.dep_, token1.head)
spacy.displacy.render(doc, style="dep", options={"distance": 100})


def get_ancestors(token):

    path = []
    current = token
    while current:
        path.append(current)
        if current.head == current:
            break
        current = current.head
    return path

def lca(token1, token2):

    ancestors1 = get_ancestors(token1)
    ancestors2 = get_ancestors(token2)
    for t1 in reversed(ancestors1):
        if t1 in ancestors2:
            return t1
    return None

# Пример использования
token1 = doc[4]
token2 = doc[8]

common_ancestor = lca(token1, token2)
print(f"Наименьший общий предок: {common_ancestor.text} ({common_ancestor.pos_}) для {token1} и {token2}")



Наименьший общий предок: ловлю (VERB) для я и отголоске


## Задание 4:сравнить сравнение zss и cos
Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами.

In [25]:
! python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [26]:
! pip install zss torch transformers scikit-learn

In [27]:
# prompt: Сравнить три пары предложений двумя методами: сравнением расстояния редактирования деревьев зависимостей (zss) и косинусной мерой между BERT-эмбеддингами. (не использовать deeppavlov)
from transformers import BertTokenizer, BertModel
import torch
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from zss import simple_distance, Node

def build_zss_tree(token):
    node = Node(token.text)
    for child in token.children:
        node.add_child(build_zss_tree(child))
    return node

def compare_sentences_zss(sentence1, sentence2, nlp):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)

    if not doc1.is_parsed or not doc2.is_parsed:
        return None # Or handle appropriately

    # Find the root of the dependency tree
    root1 = None
    for token in doc1:
        if token.dep_ == "ROOT":
            root1 = token
            break

    root2 = None
    for token in doc2:
        if token.dep_ == "ROOT":
            root2 = token
            break

    if root1 is None or root2 is None:
        return None # Or handle sentences without a clear root

    tree1 = build_zss_tree(root1)
    tree2 = build_zss_tree(root2)

    return simple_distance(tree1, tree2)

def compare_sentences_bert(sentence1, sentence2, model, tokenizer):
    inputs1 = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(sentence2, return_tensors='pt', padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
        embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]



tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

# # Load BERT model and tokenizer
# model_name = "ruBert-base" # Or a suitable Russian BERT model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# Define sentence pairs
sentence_pairs = [
    ("Кошка сидела на коврике.", "Собака лежала на полу."),
    ("Было тихо, и над деревней стоял теплый туман.", "Стоял теплый туман, и было тихо."),
    ("Иван купил яблоки в магазине.", "Магазин продал яблоки Ивану.")
]

# Perform comparisons
for s1, s2 in sentence_pairs:
    print(f"\nСравнение предложений:")
    print(f"Предложение 1: '{s1}'")
    print(f"Предложение 2: '{s2}'")

    # ZSS comparison
    zss_distance = compare_sentences_zss(s1, s2, nlp)
    if zss_distance is not None:
        print(f"Расстояние редактирования деревьев зависимостей (ZSS): {zss_distance}")
    else:
        print("Не удалось построить деревья зависимостей для ZSS сравнения.")


    # BERT comparison
    bert_similarity = compare_sentences_bert(s1, s2, model, tokenizer)
    print(f"Косинусная мера между BERT-эмбеддингами: {bert_similarity:.4f}")


ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/usr/local/lib/python3.11/dist-packages/google/protobuf/internal/__init__.py)